In [1]:
import findspark

In [2]:
findspark.init('/home/i-sip_iot/spark-3.0.1-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('LogReg').getOrCreate()

In [6]:
from pyspark.ml.classification import LogisticRegression

In [7]:
df = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [8]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [11]:
LogRegModel = LogisticRegression()

In [12]:
fitted_d = LogRegModel.fit(df)

In [16]:
fit_summary = fitted_d.summary

In [20]:
fit_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [21]:
df_train, df_test = df.randomSplit([0.7, 0.3])

In [22]:
fit_model = LogRegModel.fit(df_train)

In [23]:
prediction = fit_model.evaluate(df_test)

In [25]:
prediction.accuracy

1.0

In [26]:
prediction.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[122,123,124...|[17.9782724656987...|[0.99999998443548...|       0.0|
|  0.0|(692,[123,124,125...|[32.0330471211555...|[0.99999999999998...|       0.0|
|  0.0|(692,[123,124,125...|[33.1613355514274...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[85.7498501757283...|[1.0,5.7453070597...|       0.0|
|  0.0|(692,[124,125,126...|[31.5029366949267...|[0.99999999999997...|       0.0|
|  0.0|(692,[124,125,126...|[25.6423416695708...|[0.99999999999269...|       0.0|
|  0.0|(692,[124,125,126...|[20.2369626875005...|[0.99999999837370...|       0.0|
|  0.0|(692,[126,127,128...|[27.1315617627799...|[0.99999999999835...|       0.0|
|  0.0|(692,[126,127,128...|[27.2979804675352...|[0.99999999999860...|       0.0|
|  0.0|(692,[127

In [27]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [28]:
binary_eval = BinaryClassificationEvaluator()

In [29]:
eval_check = binary_eval.evaluate(prediction.predictions)

In [30]:
eval_check

1.0

## Work on Titanic Tittanic dataset

In [1]:
import findspark

In [2]:
findspark.init('/home/i-sip_iot/spark-3.0.1-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('titanLR').getOrCreate()

In [7]:
df = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [19]:
df.createOrReplaceGlobalTempView('Tab1')

In [20]:
query = spark.sql('SELECT * FROM global_temp.Tab1')

In [21]:
query.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [23]:
query.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [22]:
query.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [24]:
dfn = df.select(['Survived', 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [25]:
dfn.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     3|  male|null|    0|    0| 8.4583|       Q|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       

In [26]:
dfn = dfn.dropna()

In [27]:
dfn.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       S|
|       0|     3|female|14.0|    0|    0| 7.8542|       

In [28]:
from pyspark.ml.feature import (OneHotEncoder, VectorIndexer, StringIndexer, VectorAssembler)

In [29]:
idx_sex = StringIndexer(inputCol='Sex', outputCol='SexIndex')

In [30]:
Vect_sex = OneHotEncoder(inputCol='SexIndex', outputCol='SexVect')

In [31]:
idx_embark = StringIndexer(inputCol='Embarked', outputCol='EmbarkedIDX')

In [32]:
Vect_embarked = OneHotEncoder(inputCol='EmbarkedIDX', outputCol='imbarkedVect')

In [33]:
assembling = VectorAssembler(inputCols=['imbarkedVect', 'SexVect', 'Pclass', 'Age', 'SibSp', 'Fare'], outputCol='features')

In [37]:
from pyspark.ml.classification import LogisticRegression

In [38]:
from pyspark.ml.pipeline import Pipeline

In [39]:
LR_Model = LogisticRegression(featuresCol='features', labelCol='Survived', predictionCol='prediction')

In [43]:
pipeline = Pipeline(stages=[idx_sex, Vect_sex, idx_embark, Vect_embarked, assembling, LR_Model])

In [44]:
train_df, testdf = dfn.randomSplit([0.7, 0.3])

In [45]:
trained_model = pipeline.fit(train_df)

In [46]:
trained_model

PipelineModel_26bb617f280c

In [47]:
test_res = trained_model.transform(testdf)

In [48]:
test_res.show()

+--------+------+------+----+-----+-----+-------+--------+--------+-------------+-----------+-------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|SexIndex|      SexVect|EmbarkedIDX| imbarkedVect|            features|       rawPrediction|         probability|prediction|
+--------+------+------+----+-----+-----+-------+--------+--------+-------------+-----------+-------------+--------------------+--------------------+--------------------+----------+
|       0|     1|female| 2.0|    1|    2| 151.55|       S|     1.0|    (1,[],[])|        0.0|(2,[0],[1.0])|[1.0,0.0,0.0,1.0,...|[-3.8043623461529...|[0.02178810038122...|       1.0|
|       0|     1|  male|28.0|    1|    0|82.1708|       C|     0.0|(1,[0],[1.0])|        1.0|(2,[1],[1.0])|[0.0,1.0,1.0,1.0,...|[-0.2515961088384...|[0.43743068139611...|       1.0|
|       0|     1|  male|30.0|    0|    0|  27.75|       C|     0.0|(1,[0],[1.0])|        1

In [50]:
test_res.columns

['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'SexIndex',
 'SexVect',
 'EmbarkedIDX',
 'imbarkedVect',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [49]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [51]:
evaluation_ = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [54]:
test_res.select(['prediction', 'Survived']).show()

+----------+--------+
|prediction|Survived|
+----------+--------+
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       1.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       0.0|       0|
|       1.0|       0|
|       1.0|       0|
|       0.0|       0|
+----------+--------+
only showing top 20 rows



In [56]:
Accu = evaluation_.evaluate(test_res)

In [57]:
Accu

0.778403898872982